This is the last part of this project. In this section, the geocoded addresses of NYC restaurants with noted critical violations according to the Department of Mental Health and Hygiene in 2019 (restaurant address, and coordinates) saved from the 'geocode.ipynb' notebook are used for visualization.

# Import Libraries

In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import ast

# Import Data

In [2]:
# read in dataset of geocoded addresses with coordinates
data = pd.read_csv('./data/Geocoded_Addresses.csv', usecols=['ADDRESS', 'POINT'])

In [3]:
data.head()

,ADDRESS,POINT
0,469 FLATBUSH AVENUE BROOKLYN NY 11225,"(40.66293035, -73.9617257983564, 0.0)"
1,7114 AVENUE U BROOKLYN NY 11234,"(40.619892, -73.9068502510335, 0.0)"
2,1310 SURF AVENUE BROOKLYN NY 11224,"(40.57530105, -73.9814687031484, 0.0)"
3,18 WEST HOUSTON STREET MANHATTAN NY 10012,"(40.7257247, -73.9970561, 0.0)"
4,60 WALL STREET MANHATTAN NY 10005,"(40.70617305, -74.0085161961879, 0.0)"


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 2 columns):
ADDRESS    461 non-null object
POINT      461 non-null object
dtypes: object(2)
memory usage: 7.3+ KB


In [5]:
# read in data of restaurant names and addresses
restaurants = pd.read_csv('./data/NYC_Restaurant_Inspections_Data_Critical_2019.csv', usecols=['RESTAURANT NAME', 'ADDRESS'])

In [6]:
restaurants.head()

,RESTAURANT NAME,ADDRESS
0,WENDY'S,469 FLATBUSH AVENUE BROOKLYN NY 11225
1,WILKEN'S FINE FOOD,7114 AVENUE U BROOKLYN NY 11234
2,NATHAN'S FAMOUS,1310 SURF AVENUE BROOKLYN NY 11224
3,ANGELIKA FILM CENTER,18 WEST HOUSTON STREET MANHATTAN NY 10012
4,HO MEI RESTAURANT,10305 37 AVENUE QUEENS NY 11368


In [7]:
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9620 entries, 0 to 9619
Data columns (total 2 columns):
RESTAURANT NAME    9620 non-null object
ADDRESS            9620 non-null object
dtypes: object(2)
memory usage: 150.4+ KB


# Data Wrangling

In [8]:
# check an individual value in POINT column to see how it is formatted - notice it is a string
data['POINT'][0]

'(40.66293035, -73.9617257983564, 0.0)'

In [9]:
# de-string the values in the POINT column
data['POINT'] = data['POINT'].apply(lambda x: ast.literal_eval(x))

In [10]:
# check the format of the individual value again
data['POINT'][0]

(40.66293035, -73.9617257983564, 0.0)

In [11]:
# split the POINT column into 3 separate columns for each coordinate
data[['X', 'Y', 'Z']] = pd.DataFrame(data['POINT'].tolist(), index=data.index)

In [12]:
data.head()

,ADDRESS,POINT,X,Y,Z
0,469 FLATBUSH AVENUE BROOKLYN NY 11225,"(40.66293035, -73.9617257983564, 0.0)",40.662930,-73.961726,0.0
1,7114 AVENUE U BROOKLYN NY 11234,"(40.619892, -73.9068502510335, 0.0)",40.619892,-73.906850,0.0
2,1310 SURF AVENUE BROOKLYN NY 11224,"(40.57530105, -73.9814687031484, 0.0)",40.575301,-73.981469,0.0
3,18 WEST HOUSTON STREET MANHATTAN NY 10012,"(40.7257247, -73.9970561, 0.0)",40.725725,-73.997056,0.0
4,60 WALL STREET MANHATTAN NY 10005,"(40.70617305, -74.0085161961879, 0.0)",40.706173,-74.008516,0.0


In [13]:
# look at the value counts of column Z. 
data.Z.value_counts()

0.0    461
Name: Z, dtype: int64

In [14]:
# drop column Z since they are all 0s here
data.drop('Z', axis=1, inplace=True)

In [15]:
data.head()

,ADDRESS,POINT,X,Y
0,469 FLATBUSH AVENUE BROOKLYN NY 11225,"(40.66293035, -73.9617257983564, 0.0)",40.662930,-73.961726
1,7114 AVENUE U BROOKLYN NY 11234,"(40.619892, -73.9068502510335, 0.0)",40.619892,-73.906850
2,1310 SURF AVENUE BROOKLYN NY 11224,"(40.57530105, -73.9814687031484, 0.0)",40.575301,-73.981469
3,18 WEST HOUSTON STREET MANHATTAN NY 10012,"(40.7257247, -73.9970561, 0.0)",40.725725,-73.997056
4,60 WALL STREET MANHATTAN NY 10005,"(40.70617305, -74.0085161961879, 0.0)",40.706173,-74.008516


In [16]:
# merge restaurant dataset to geocoded data so that final dataset has restuarant names with coordinates and addresses
locations = data.merge(restaurants, on='ADDRESS', how='left')

In [17]:
locations.shape

(486, 5)

In [18]:
locations.head()

,ADDRESS,POINT,X,Y,RESTAURANT NAME
0,469 FLATBUSH AVENUE BROOKLYN NY 11225,"(40.66293035, -73.9617257983564, 0.0)",40.662930,-73.961726,WENDY'S
1,7114 AVENUE U BROOKLYN NY 11234,"(40.619892, -73.9068502510335, 0.0)",40.619892,-73.906850,WILKEN'S FINE FOOD
2,1310 SURF AVENUE BROOKLYN NY 11224,"(40.57530105, -73.9814687031484, 0.0)",40.575301,-73.981469,NATHAN'S FAMOUS
3,18 WEST HOUSTON STREET MANHATTAN NY 10012,"(40.7257247, -73.9970561, 0.0)",40.725725,-73.997056,ANGELIKA FILM CENTER
4,60 WALL STREET MANHATTAN NY 10005,"(40.70617305, -74.0085161961879, 0.0)",40.706173,-74.008516,THE COUNTRY CAFE


In [19]:
locations.tail()

,ADDRESS,POINT,X,Y,RESTAURANT NAME
481,160 BROADWAY MANHATTAN NY 10038,"(40.70937965, -74.0100296338982, 0.0)",40.709380,-74.010030,MILK N' HONEY
482,917 MANOR ROAD STATEN ISLAND NY 10314,"(40.60439225, -74.1205629156551, 0.0)",40.604392,-74.120563,MANOR HOUSE RESTAURANT
483,129 GATES AVENUE BROOKLYN NY 11238,"(40.6849008, -73.9629318, 0.0)",40.684901,-73.962932,(LEWIS DRUG STORE) LOCANDA VINI E OLII
484,132 CROSBY STREET MANHATTAN NY 10012,"(40.7247777, -73.996323, 0.0)",40.724778,-73.996323,LAHORE DELICATESSEN
485,802-804 KINGS HIGHWAY BROOKLYN NY 11223,"(40.6064537755102, -73.9644138979592, 0.0)",40.606454,-73.964414,CAFE RENAISSANCE


In [20]:
# check for null values
locations.isnull().sum()

ADDRESS            0
POINT              0
X                  0
Y                  0
RESTAURANT NAME    0
dtype: int64

# Visualization

In this section, the coordinates are visualized on a map using Folium, which is an interactive mapping library built on leaflet.js. The coordinates are first visualized individually on the map and then they are visualized in clusters on the map. Both versions give a slightly different view of the restaurants. 

### Individual Points

This version of the map show at a glance the areas with most or least restaurants with noted critical violations in 2019.

In [21]:
# create a map with coordinates for NYC
m = folium.Map(
                        location = [40.7230018,-73.9911482], 
                        tiles = 'Stamen Toner',
                        zoom_start=11)


In [22]:
# add all coordinates to map
locations.apply(lambda row: folium.CircleMarker(
                                location=[row['X'], row['Y']], 
                                radius=10, 
                                popup=row['RESTAURANT NAME'], 
                                color='red', 
                                fill=True, 
                                fill_color='red').add_to(m), 
                        axis=1)

0      <folium.vector_layers.CircleMarker object at 0...
1      <folium.vector_layers.CircleMarker object at 0...
2      <folium.vector_layers.CircleMarker object at 0...
3      <folium.vector_layers.CircleMarker object at 0...
4      <folium.vector_layers.CircleMarker object at 0...
5      <folium.vector_layers.CircleMarker object at 0...
6      <folium.vector_layers.CircleMarker object at 0...
7      <folium.vector_layers.CircleMarker object at 0...
8      <folium.vector_layers.CircleMarker object at 0...
9      <folium.vector_layers.CircleMarker object at 0...
10     <folium.vector_layers.CircleMarker object at 0...
11     <folium.vector_layers.CircleMarker object at 0...
12     <folium.vector_layers.CircleMarker object at 0...
13     <folium.vector_layers.CircleMarker object at 0...
14     <folium.vector_layers.CircleMarker object at 0...
15     <folium.vector_layers.CircleMarker object at 0...
16     <folium.vector_layers.CircleMarker object at 0...
17     <folium.vector_layers.Ci

In [23]:
m

In [24]:
# save map to file
# m.save('./images/inspections_map_individual.html')
# m.save('./images/inspections_map_individual.png')

### Clustered Points

This version of the map show in a more condensed and cleaner manner the areas with most or least restaurants with noted critical violations in 2019.

In [25]:
# create a map with coordinates for NYC
mp = folium.Map(
                        location = [40.7230018,-73.9911482],
                        zoom_start=11)

# customize basemap to be CartoDB Dark Matter
folium.TileLayer('cartodbdark_matter').add_to(mp)

In [26]:
# create cluster and add to map
cluster = MarkerCluster().add_to(mp)

In [27]:
# add all coordinates to cluster
locations.apply(lambda row: folium.Marker(
                                location=[row['X'], row['Y']],
                                popup = "{}, \n {}".format(row['RESTAURANT NAME'], row['ADDRESS']),
                                color = 'red',
                                fill_color = 'red').add_to(cluster), axis=1)

0      <folium.map.Marker object at 0x1204cdc88>
1      <folium.map.Marker object at 0x1204cde48>
2      <folium.map.Marker object at 0x1211e1128>
3      <folium.map.Marker object at 0x10fba2b70>
4      <folium.map.Marker object at 0x10efaa8d0>
5      <folium.map.Marker object at 0x10efaa3c8>
6      <folium.map.Marker object at 0x10efaa438>
7      <folium.map.Marker object at 0x10ebc6358>
8      <folium.map.Marker object at 0x10eb9ae48>
9      <folium.map.Marker object at 0x1204cdeb8>
10     <folium.map.Marker object at 0x1204cdef0>
11     <folium.map.Marker object at 0x1204e3198>
12     <folium.map.Marker object at 0x1204e32e8>
13     <folium.map.Marker object at 0x1204e3438>
14     <folium.map.Marker object at 0x1204e3588>
15     <folium.map.Marker object at 0x1204e36d8>
16     <folium.map.Marker object at 0x1204e3828>
17     <folium.map.Marker object at 0x1204e3978>
18     <folium.map.Marker object at 0x1204e3ac8>
19     <folium.map.Marker object at 0x1204e3c18>
20     <folium.map.M

In [28]:
mp

In [29]:
# save map to file
# mp.save('./images/inspections_map_clustered.html')
# mp.save('./images/inspections_map_clustered.png')